In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import tqdm

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import exp
from scipy.optimize import curve_fit

import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
import hdf5storage
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.cm as cm

from utility_simulated_drift import (make_shifted_templates,
                                      make_default_template,
                                      draw_template,
                                      gaus,
                                      make_drift_template,
                                      shift_templates_all,
                                      WaveForms,
                                      generate_poisson_uniform_firingrate2,
                                      select_ground_truth_units,
                                      visualize_drift,
                                      generate_synthetic_data,
                                      visualize_traces,
                                      make_full_data)

colors = [
'black','grey','brown','slategrey',    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
clrs= colors


Autosaving every 180 seconds


/home/cat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# select working directory location
root_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/synthetic_drift/run10/'
try:
    os.mkdir(root_dir)
except:
    pass

# set radius of channels to consider from maximum channel outwards (in micrometers)
radius = 200

# select ground truth templates to be used
fname_templates = '/media/cat/12TB/dbox/Dropbox/code/neuropixels/allen_inst_2ndrec/templates_reloaded.npy'
geom = np.loadtxt('/home/cat/p1_g0_t0.imec0.ap_geom_384chans.txt')

# select largest X templates to be used for injection
n_templates = 100
(units, templates_ground_truth) = select_ground_truth_units(root_dir, 
                                                            fname_templates,
                                                            n_templates,
                                                            geom)

print ("templates ground truth: ", templates_ground_truth.shape)

ground truth directory already made
Total templates avialable;  (575, 101, 384)
Total units selected:  100 , ids:  [420 339 421 348 473 418  62 362 565 239] ...
templates ground truth:  (100, 101, 384)


In [3]:
# visualize the desired shifts on a particular unit
shifts = np.arange(0, 1., 0.05)

# visualize drifted template for any particular unit 
unit_id = 10
visualize_drift(shifts, geom, templates_ground_truth, unit_id,
               radius)


In [3]:
# generate synthetic drift data
sample_rate = 30000
rec_len_sec = 1800
rec_length_sample_times = rec_len_sec * sample_rate

# spikes, scales = generate_poisson_uniform_firingrate2(rec_length)
# setup shifts
shifts = np.arange(0, 1.0, 0.05)
print ("shifts: ", shifts)

data_synthetic = generate_synthetic_data(root_dir,
                                         rec_len_sec, 
                                         sample_rate, 
                                         shifts, 
                                         units, 
                                         templates_ground_truth,
                                         radius,
                                         geom)

np.savetxt(root_dir+'/ground_truth/shifts.txt', shifts)
print (data_synthetic.shape)


shifts:  [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95]
(54000000, 384)
[       0.          2842105.26315789  5684210.52631579  8526315.78947368
 11368421.05263158 14210526.31578947 17052631.57894737 19894736.84210526
 22736842.10526316 25578947.36842106 28421052.63157895 31263157.89473684
 34105263.15789474 36947368.42105263 39789473.68421052 42631578.94736842
 45473684.21052632 48315789.47368421 51157894.73684211 54000000.        ]
window:  [0.0, 2842105.263157895] , shift:  0.0  (inter channel units)
 inserting unit:  0 0
 inserting unit:  50 50
************


window:  [2842105.263157895, 5684210.52631579] , shift:  0.05  (inter channel units)
 inserting unit:  0 0
 inserting unit:  50 50
************


window:  [5684210.52631579, 8526315.789473685] , shift:  0.1  (inter channel units)
 inserting unit:  0 0
 inserting unit:  50 50
************


window:  [8526315.789473685, 11368421.05263158] , shift:  0.15000000000000002  (int

In [4]:
# visualize the final traces
visualize_traces(data_synthetic, 0, 5000)


In [5]:
# Make a correlated data chunk of 10 seconds 
# longer takes too long of time; shorter will add more boundary artifcats to recording
time = 10*sample_rate
print ("length of correlated noise snipit: ", time)

# use ground_truth_templates to make noise
temps = templates_ground_truth.copy()
WF = WaveForms(temps.transpose(0,2,1))
correlated_noise = WF.generate_correlated_noise_add(time)

print ("DONE MAKING CORRELATED NOISE for 10 sec chunk")


length of correlated noise snipit:  300000
self.n_unit, self.n_channel, self.n_time:  100 384 101
chan:  0
chan:  50
chan:  100
chan:  150
chan:  200
chan:  250
chan:  300
chan:  350
DONE MAKING CORRELATED NOISE for 10 sec chunk


In [6]:
# blend correlated noise with injected spike dataset
data_sum = make_full_data(data_synthetic, rec_length_sample_times, 
                          sample_rate, root_dir,
                          correlated_noise)


white noise:  (300000, 384)
time chunk:  0  to  300000
time chunk:  300000  to  600000
time chunk:  600000  to  900000
time chunk:  900000  to  1200000
time chunk:  1200000  to  1500000
time chunk:  1500000  to  1800000
time chunk:  1800000  to  2100000
time chunk:  2100000  to  2400000
time chunk:  2400000  to  2700000
time chunk:  2700000  to  3000000
time chunk:  3000000  to  3300000
time chunk:  3300000  to  3600000
time chunk:  3600000  to  3900000
time chunk:  3900000  to  4200000
time chunk:  4200000  to  4500000
time chunk:  4500000  to  4800000
time chunk:  4800000  to  5100000
time chunk:  5100000  to  5400000
time chunk:  5400000  to  5700000
time chunk:  5700000  to  6000000
time chunk:  6000000  to  6300000
time chunk:  6300000  to  6600000
time chunk:  6600000  to  6900000
time chunk:  6900000  to  7200000
time chunk:  7200000  to  7500000
time chunk:  7500000  to  7800000
time chunk:  7800000  to  8100000
time chunk:  8100000  to  8400000
time chunk:  8400000  to  870000

In [12]:
# visualize first 10000 time points;
visualize_traces(data_sum, 0, 10000)
